In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
cd drive/My\ Drive/Fake\ Orders/

In [0]:
ls

## Data Processing

In [0]:
# Delete unnecessary columns and add 'time group', 'weekday', 'is_holiday', 'locate_id' as new columns


In [0]:
# Combine space & order information

In [0]:
# Transpose 'space_device' and save the results in space & order

## Fake Order Generation

In [0]:
import csv
import datetime
##### Part 1: Read the spaces information #####

with open('20191226_spaces_info.csv', newline='', encoding='big5') as space_file:

  # 讀取 CSV 檔內容，將每一列轉成一個 dictionary
  spaces_info = csv.DictReader(space_file)
  
  # 以迴圈讀取指定欄位
  locate_id = []; space_id = []; price = []; size = []; zipcode = []; updated_at = []; enabled = [];
  open_day = []; close_day = []; open_time = []; close_time = []; creat_time = []; mrt = [];
  for i in spaces_info:
    locate_id.append(i['locate_id'])
    space_id.append(i['space_id'])
    price.append(i['price'])
    size.append(i['size'])
    zipcode.append(i['zipcode'])
    open_day.append(i['open_day'])
    close_day.append(i['close_day'])
    open_time.append(i['open_time'])
    close_time.append(i['close_time'])
    creat_time.append(i['creat_time'])
    mrt.append(i['mrt'])
    updated_at.append(i['updated_at']) # 20191228_updated
    enabled.append(i['enabled'])       # 20191228_updated
  space_file.close()

locate_id = list(map(int, locate_id))
space_id = list(map(int, space_id))
price = list(map(int, price))
size = list(map(int, size))
zipcode = list(map(int, zipcode))
open_day = list(map(int, open_day))
close_day = list(map(int, close_day))
enabled = list(map(int, enabled))

print("Read the Space information successfully.")


##### Read the real Orders information #####

with open('2019.12.28 old_new_order_cat1_change_timeformat.csv', newline='', encoding='big5') as test_file:

  # 讀取 CSV 檔內容，將每一列轉成一個 dictionary
  test_info = csv.DictReader(test_file)
  
  # 以迴圈讀取指定欄位
  r_space_id = []; r_date = []; r_start = []; r_end = [];
  r_start_time_group = []; r_end_time_group = [];
  for i in test_info:
    r_space_id.append(i['space_id'])
    r_date.append(i['date'])
    r_start.append(i['start'])
    r_end.append(i['end'])
    r_start_time_group.append(i['start_time_group'])
    r_end_time_group.append(i['End_time_group'])
  test_file.close()

r_space_id = list(map(int, r_space_id))
r_no_dupl_space_id = list(set(r_space_id)) #delete the duplicates & use set() to sort the 
r_start_time_group = list(map(int, r_start_time_group))
r_end_time_group = list(map(int, r_end_time_group))

for i in range(0, len(r_date)):
  r_date[i] = datetime.datetime.strptime(r_date[i], "%d/%m/%Y")

print("Read the Real Order information successfully.")


In [0]:
##### Part 2: Generate Fake Orders => in Current Real Orders Time #####

with open('Fake_Orders_in_Real_Orders_Time.csv', mode='w', newline='', encoding='big5') as f1:
    FOrders_writer = csv.writer(f1)

    order_id = 300000    #訂單編號
    category = 1         #普通方案
    Is_holiday = 0       #是否為假日
    status = 1           #沒租出去

    start_time_group = 0
    end_time_group = 0
    morning = datetime.datetime.strptime("12:00", format)     # 1: 06:00 ~ 12:00
    noon = datetime.datetime.strptime("14:00", format)        # 2: 12:00 ~ 14:00
    afternoon = datetime.datetime.strptime("18:00", format)   # 3: 14:00 ~ 18:00
    night = datetime.datetime.strptime("23:59", format)       # 4: 18:00 ~ 24:00 , 00:00 ~ 06:00
    midnight = datetime.datetime.strptime("06:00", format)

    ##### Write the Fake Orders in real order time #####
    FOrders_writer.writerow(['id', 'locate_id', 'space_id', 'hours', 'price', 'order_price', 'size', 'price_per_size', 'date', 'start', 'end', 'start_time_group', 'end_time_group', 'weekday', 'Is_holiday', 'zipcode', 'category', 'mrt', 'status'])
    total = 4
    count_temp = 0
    len_count = 0
    for i in range(0, len(r_no_dupl_space_id)):  #遍歷真實訂單中的space_id（不重複）
      t = i
      while(r_no_dupl_space_id[i] != space_id[t]):
        t += 1 

      ##### 計算space的創造時間 ######
      len_of_same_space_order = r_space_id.count(r_space_id[len_count]) #計算同一間房間有多少訂單
      len_count += len_of_same_space_order
      
      same_space_time_group = []
      temp_compare_time_group = []

      ##### 設定房間起始日期 #####
      order_start = datetime.datetime.strptime(open_time[t], format) #訂單開始時間
      order_end = datetime.datetime.strptime(close_time[t], format)  #訂單結束時間
      if (order_start >= midnight and order_start < morning):
        temp_compare_time_group.append(1)
      elif (order_start >= morning and order_start < noon):
        temp_compare_time_group.append(2)
      elif (order_start >= noon and order_start < afternoon):
        temp_compare_time_group.append(3)
      else:
        temp_compare_time_group.append(4)
      
      if (order_end > midnight and order_end <= morning):
        temp_compare_time_group.append(1)
      elif (order_end > morning and order_end <= noon):
        temp_compare_time_group.append(2)
      elif (order_end > noon and order_end <= afternoon):
        temp_compare_time_group.append(3)
      else:
        temp_compare_time_group.append(4)
      temp_compare_time_group = list(range(temp_compare_time_group[0], temp_compare_time_group[1] + 1))
      #temp_compare_time_group = [1, 2, 3, 4]

      for g in range(0, len_of_same_space_order):
        if((count_temp + 1) == len(r_date)):
          r_date.append('')
        if(r_date[count_temp] == r_date[count_temp+1]):
          for u in range(r_start_time_group[count_temp], r_end_time_group[count_temp] + 1):
            same_space_time_group.append(u)

        elif(r_date[count_temp] != r_date[count_temp+1]):
          for u in range(r_start_time_group[count_temp], r_end_time_group[count_temp] + 1):
            same_space_time_group.append(u)

          diff_time_group = list(set(temp_compare_time_group) - set(same_space_time_group))
          for k in diff_time_group:
            if (k == 1):
              order_start = datetime.datetime.strptime("09:00", format)
              order_end = morning
            elif (k == 2):
              order_start = morning
              order_end = noon
            elif (k == 3):
              order_start = noon
              order_end = afternoon
            else:
              order_start = afternoon
              order_end = datetime.datetime.strptime("21:00", format)
            ##### 計算訂單租用時間 #####
            diff = order_end - order_start 
            seconds = diff.total_seconds()
            hours = seconds // 3600
            minutes = (seconds % 3600) // 60
            duration = hours + minutes/60  #訂單租用時間
            Is_holiday = 1 if(r_date[count_temp].isoweekday() == 6 or r_date[count_temp].isoweekday() == 7) else 0
            ##### 開始產生訂單 #####
            FOrders_writer.writerow([order_id, locate_id[t], space_id[t], duration, price[t], price[t]*duration, size[t], price[t]/size[t], r_date[count_temp].date().strftime("%Y/%m/%d"), order_start.strftime("%H:%M:%S"), order_end.strftime("%H:%M:%S"), k, k, r_date[count_temp].isoweekday(), Is_holiday, zipcode[t], category, mrt[t], status])
            order_id += 1
          same_space_time_group = []
        count_temp += 1
    print("Generate the Fake Orders successfully.")
    f1.close()

##### Part 3: Generate Fake Orders => in Empty Time #####

format = '%H:%M'
##### Generate the Fake Orders #####
with open('Fake_Orders_in_Empty_Time.csv', mode='w', newline='', encoding='big5') as f2:
  FOrders_writer = csv.writer(f2)

  order_id = 600000    #假訂單編號
  category = 1         #普通方案
  Is_holiday = 0       #是否為假日
  status = 1           #沒有真實訂單

  #start_date = "08/02/2017"  #str %d/%m/%Y
  #start_date = datetime.datetime.strptime(start_date, "%d/%m/%Y")
  end_date = "23/12/2019"    
  end_date = datetime.datetime.strptime(end_date , "%d/%m/%Y")

  # end_date = "2019/12/23"    
  # end_date = datetime.datetime.strptime(end_date , "%Y/%m/%d")

  # len_of_date = (end_date - date).days + 1

  check_day = []
  start_time_group = 0
  end_time_group = 0
  morning = datetime.datetime.strptime("12:00", format)     # 1: 06:00 ~ 12:00
  noon = datetime.datetime.strptime("14:00", format)        # 2: 12:00 ~ 14:00
  afternoon = datetime.datetime.strptime("18:00", format)   # 3: 14:00 ~ 18:00
  night = datetime.datetime.strptime("23:59", format)       # 4: 18:00 ~ 24:00 , 00:00 ~ 06:00
  midnight = datetime.datetime.strptime("06:00", format)

  ##### Write the Fake Orders #####
  FOrders_writer.writerow(['id', 'locate_id', 'space_id', 'size', 'hours', 'price', 'order_price', 'price_per_size', 'date', 'start', 'end', 'start_time_group', 'end_time_group', 'weekday', 'Is_holiday', 'zipcode', 'category', 'mrt', 'status'])
  for i in range(0, len(space_id)):  #space_list

    ##### 計算space的創造時間 ######
    space_creat = datetime.datetime.strptime(creat_time[i], '%d%b%y:%H:%M:%S') #read info like '08FEB17:21:43:04'
    space_creat_date = datetime.datetime.strptime(space_creat.strftime("%d/%m/%Y"), "%d/%m/%Y") #空間建立時間
    
    if(enabled[i] == 0):
      space_end = datetime.datetime.strptime(updated_at[i], '%d%b%y:%H:%M:%S')   #read info like '23APR19:01:20:42'
      space_end_date = datetime.datetime.strptime(space_end.strftime("%d/%m/%Y") , "%d/%m/%Y")
      len_of_date = (space_end_date - space_creat_date).days + 1
    else:
      len_of_date = (end_date - space_creat_date).days + 1 #int 計算兩個日期間距
    
    ##### 設定房間起始日期 #####
    date = datetime.datetime.strptime(space_creat.strftime("%d/%m/%Y"), "%d/%m/%Y")
    order_start = datetime.datetime.strptime(open_time[i], format) #訂單開始時間
    order_end = datetime.datetime.strptime(close_time[i], format)  #訂單結束時間
    if (order_start >= midnight and order_start < morning):
      start_time_group = 1
    elif (order_start >= morning and order_start < noon):
      start_time_group = 2
    elif (order_start >= noon and order_start < afternoon):
      start_time_group = 3
    else:
      start_time_group = 4

    if (order_end > midnight and order_end <= morning):
      end_time_group = 1
    elif (order_end > morning and order_end <= noon):
      end_time_group = 2
    elif (order_end > noon and order_end <= afternoon):
      end_time_group = 3
    else:
      end_time_group = 4

    ##### 開始產生訂單 #####
    for j in range(0, len_of_date): #根據空間的建立時間，再跑到 31/12/2019
      start = open_time[i]  #重置每日的租用時段
      check_day = list(range(open_day[i], close_day[i]+1))
      
      Is_holiday = 1 if(date.isoweekday() == 6 or date.isoweekday() == 7) else 0

      if (date.isoweekday() in check_day):
        for k in range(start_time_group, end_time_group + 1):
          if (k == 1):
            order_start = datetime.datetime.strptime("09:00", format)
            order_end = morning
          elif (k == 2):
            order_start = morning
            order_end = noon
          elif (k == 3):
            order_start = noon
            order_end = afternoon
          else:
            order_start = afternoon
            order_end = datetime.datetime.strptime("21:00", format)
          ##### 計算訂單租用時間 #####
          diff = order_end - order_start 
          seconds = diff.total_seconds()
          hours = seconds // 3600
          minutes = (seconds % 3600) // 60
          duration = hours + minutes/60  #訂單租用時間
          
          ##### Actual write #####
          FOrders_writer.writerow([order_id, locate_id[i], space_id[i], size[i], duration, price[i], price[i]*duration, price[i]/size[i], date.date().strftime("%Y/%m/%d"), order_start.strftime("%H:%M:%S"), order_end.strftime("%H:%M:%S"), k, k, date.isoweekday(), Is_holiday, zipcode[i], category, mrt[i], status])
          order_id += 1
      date = date + datetime.timedelta(days=1)
  print("Generate the Fake Orders successfully.")
  f2.close()


### Fake & Real Orders: Combine and Filter Overlapping Parts

In [0]:
# Part 1: Combine Data 

In [0]:
# Part 2: Filter Overlapping Parts

## Model Trianing

In [0]:
# 將資料套入 Regression model 進行預測

## Price Predicting & Setting

In [0]:
# Part 1: Price Predicting

In [0]:
# Part 2: Price Setting